In [1]:
from tqdm import tqdm
from sklearn.decomposition import NMF
import pandas as pd
from scipy import sparse
import numpy as np
import implicit

# read data
ratings = pd.read_feather('data/amazon_review_ratings.feather')

# indexing ids
# userid
userid_unique = pd.Series(ratings["userId"].unique())
index_userid_dict = userid_unique.to_dict()
# inverse
userid_index_dict = dict(map(reversed, index_userid_dict.items()))

# itemid
itemid_unique = pd.Series(ratings["itemId"].unique())
index_itemid_dict = itemid_unique.to_dict()
# inverse
itemid_index_dict = dict(map(reversed, index_itemid_dict.items()))

ratings["userId_reindex"] = ratings["userId"].map(userid_index_dict)
ratings["itemid_reindex"] = ratings["itemId"].map(itemid_index_dict)

# reindexしたidを使って、アイテムとジャンルの対応が取れるdictを作る
itemid_genres_dict = ratings[['itemid_reindex', 'category']].set_index('itemid_reindex')['category'].to_dict()

In [2]:
import cloudpickle
X_train = cloudpickle.load(open("output/Amazon-X_train.pkl","rb"))
X_test = cloudpickle.load(open("output/Amazon-X_test.pkl","rb"))
test_movies_and_TVs_pos_items_dict = cloudpickle.load(open('output/test_movies_and_TVs_pos_items_dict.pkl', 'rb'))

In [3]:
# aggregateのtrainをbookとmoviesに分離する
# bookの列
book_columns = [v for v in range(X_train.shape[1]) if 'book' in itemid_genres_dict[v]]
# moviesの列
movies_columns = [v for v in range(X_train.shape[1]) if 'movies_and_TVs' in itemid_genres_dict[v]]

# 選んだカラムに応じてとってくる
book_train = X_train[:, book_columns]
movies_train = X_train[:, movies_columns]

# moviesのみ、アイテムidのconcatとの対応関係が必要なので辞書として持っておく
movies_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'movies_and_TVs' in itemid_genres_dict[v]:
        movies_concat_itemid_dict[v] = count
        count += 1

In [4]:
# アイテムidのconcatとの対応関係が必要なので辞書として持っておく
book_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'book' in itemid_genres_dict[v]:
        book_concat_itemid_dict[v] = count
        count += 1
# inverse
inverse_book_concat_itemid_dict = dict(map(reversed, book_concat_itemid_dict.items()))

movies_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'movies_and_TVs' in itemid_genres_dict[v]:
        movies_concat_itemid_dict[v] = count
        count += 1
# inverse
inverse_movies_concat_itemid_dict = dict(map(reversed, movies_concat_itemid_dict.items()))

In [5]:
# それぞれにアクションしていないユーザを削る
# 全ユーザと、削ったあとでの対応関係を辞書として持っておく
book_train_selected = book_train[book_train.getnnz(1)>0]
movies_train_selected = movies_train[movies_train.getnnz(1)>0]

book_train_action_users = {}
book_users = book_train.getnnz(1)>0
count = 0
for i in range(book_train.shape[0]):
    if book_users[i]:
        book_train_action_users[i] = count
        count += 1

# inverse
inverse_book_train_action_users = dict(map(reversed, book_train_action_users.items()))

movies_train_action_users = {}
movies_users = movies_train.getnnz(1)>0
count = 0
for i in range(movies_train.shape[0]):
    if movies_users[i]:
        movies_train_action_users[i] = count
        count += 1

# inverse
inverse_movies_train_action_users = dict(map(reversed, movies_train_action_users.items()))

In [8]:
# それぞれでALSする
# 今回は mediateでやったときのものを使う
book_ALS = cloudpickle.load(open('output/book_ALS.pkl', 'rb'))
movies_ALS = cloudpickle.load(open("output/movies_ALS.pkl","rb"))

book_ALS_user_vectors = book_ALS.user_factors
movies_ALS_user_vectors = movies_ALS.user_factors

In [9]:
# bookとmoviesでoverlapしているユーザで、ベクトルの対応表を作る
overlap_book_user_vectors = []
overlap_movies_user_vectors = []
count = 0
for u in tqdm(range(X_train.shape[0])):
    if u in book_train_action_users and u in movies_train_action_users:
        overlap_book_user_vectors.append(book_ALS_user_vectors[book_train_action_users[u]].tolist())
        overlap_movies_user_vectors.append(movies_ALS_user_vectors[movies_train_action_users[u]].tolist())

100%|██████████| 690240/690240 [00:00<00:00, 917207.17it/s]


In [32]:
# AutoEncoderの学習をする(movielensで良かったモデルがAutoEncoderだったので、こちらではこれに絞る)
from keras.layers import Input, Dense, Dropout
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

def build_model(input_dim, output_dim):
    inputs = Input(shape=(input_dim,))
    encoded = Dense(128, activation='relu')(inputs)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dense(32, activation='relu')(encoded)

    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(output_dim, activation='sigmoid')(decoded)
    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae','mse'])
    return autoencoder

In [33]:
from sklearn.model_selection import train_test_split
X_train_vector, X_test_vector, y_train_vector, y_test_vector = train_test_split(overlap_book_user_vectors, overlap_movies_user_vectors, random_state=42)
X_train_vector, X_val_vector, y_train_vector, y_val_vector = train_test_split(X_train_vector, y_train_vector, random_state=42)

epoch_size = 100
batch_size = 256
mcheck = ModelCheckpoint(
    'output/amazon-model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1)
es_cb = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    mode='auto')
model = build_model(np.array(X_train_vector).shape[1], np.array(y_train_vector).shape[1])
model.fit(
    np.array(X_train_vector),
    np.array(y_train_vector),
    batch_size=batch_size,
    epochs=epoch_size,
    validation_data=(
        np.array(X_val_vector),
        np.array(y_val_vector)),
    callbacks=[
        mcheck,
        es_cb],
    shuffle=True,
    verbose=1)

Train on 11525 samples, validate on 3842 samples
Epoch 1/100
11525/11525 [==============================] - 1s 47us/step - loss: 0.1620 - mae: 0.3532 - mse: 0.1620 - val_loss: 0.0126 - val_mae: 0.0466 - val_mse: 0.0126

Epoch 00001: val_loss improved from inf to 0.01262, saving model to output/amazon-model.h5
Epoch 2/100
11525/11525 [==============================] - 0s 18us/step - loss: 0.0122 - mae: 0.0446 - mse: 0.0122 - val_loss: 0.0125 - val_mae: 0.0442 - val_mse: 0.0125

Epoch 00002: val_loss improved from 0.01262 to 0.01251, saving model to output/amazon-model.h5
Epoch 3/100
11525/11525 [==============================] - 0s 17us/step - loss: 0.0122 - mae: 0.0444 - mse: 0.0122 - val_loss: 0.0125 - val_mae: 0.0442 - val_mse: 0.0125

Epoch 00003: val_loss did not improve from 0.01251
Epoch 4/100
11525/11525 [==============================] - 0s 16us/step - loss: 0.0122 - mae: 0.0444 - mse: 0.0122 - val_loss: 0.0125 - val_mae: 0.0442 - val_mse: 0.0125

Epoch 00004: val_loss did not 

In [34]:
# テストデータに対するRMSE計算
from sklearn.metrics import mean_squared_error
best_model = load_model('output/amazon-model.h5')
y_pred = best_model.predict(np.array(X_test_vector))
rmse_ = np.sqrt(mean_squared_error(y_pred, np.array(y_test_vector)))
print('rmse: {}'.format(rmse_))

rmse: 0.11936578717966138


In [35]:
# moviesのitemのベクトル
movies_item_vectors = movies_ALS.item_factors

In [36]:
from lib.recommend_util import ndcg
ndcgs = {
    'ndcg5':  [],
    'ndcg10':  [],
    'ndcg20':  [],
    'ndcg50':  [],
    'ndcg100':  []
}
count = 0
for userid, pos_items in tqdm(test_movies_and_TVs_pos_items_dict.items()):
   # pos_itemsをmovies_matrixの次元に変換する
    pos_items = np.array([movies_concat_itemid_dict[v] for v in pos_items])
    # useridに対応するユーザベクトル(book)を得る
    try:
        book_userid = book_train_action_users[userid]
        book_user_vector = book_ALS_user_vectors[book_userid, :]
        # AutoEncoderを使ってmoviesの次元に変換する
        movies_user_vector_action_AE = best_model.predict(book_user_vector.reshape(1, -1))
        # adventureのitemのベクトルと掛け合わせる
        movies_predict = np.dot(movies_user_vector_action_AE, movies_item_vectors.T)
        # sum_ratingsをargsort
        sorted_indices = np.array([v for v in np.argsort(-movies_predict)])[0]
        ndcgs['ndcg5'].append(ndcg(sorted_indices[:5], pos_items))
        ndcgs['ndcg10'].append(ndcg(sorted_indices[:10], pos_items))
        ndcgs['ndcg20'].append(ndcg(sorted_indices[:20], pos_items))
        ndcgs['ndcg50'].append(ndcg(sorted_indices[:50], pos_items))
        ndcgs['ndcg100'].append(ndcg(sorted_indices[:100], pos_items))
    except:
        count += 1
        # 推薦できないユーザの場合は無条件で0を入れる
        ndcgs['ndcg5'].append(0)
        ndcgs['ndcg10'].append(0)
        ndcgs['ndcg20'].append(0)
        ndcgs['ndcg50'].append(0)
        ndcgs['ndcg100'].append(0)

100%|██████████| 10250/10250 [01:02<00:00, 163.68it/s]


In [37]:
print("ndcg@5: {}".format(np.mean(ndcgs['ndcg5'])))
print("ndcg@10: {}".format(np.mean(ndcgs['ndcg10'])))
print("ndcg@20: {}".format(np.mean(ndcgs['ndcg20'])))
print("ndcg@50: {}".format(np.mean(ndcgs['ndcg50'])))
print("ndcg@100: {}".format(np.mean(ndcgs['ndcg100'])))

ndcg@5: 0.0040366900247044675
ndcg@10: 0.006191071032932619
ndcg@20: 0.008574333528193033
ndcg@50: 0.013658642792483655
ndcg@100: 0.018854547775509517
